# Práctica I - Módulo I - Diplomado Ciencia de Datos  
### Hernández González Ricardo Paramont  
  
**DataSets**: spotify_practica.csv , zips_practica.csv  

***

 ### 1. De los datos proporcionados realiza los siguientes puntos:  
   
   
 <div style="text-align: justify"> 
(a) Realiza el etiquetado de las variables de acuerdo a su tipo de variable. <br\>
(b) Eliminar duplicados. <br\>
(c) Completitud de las variables. <br\>
(d) Elimina las variables que tengan el 20 % o más de valores ausentes. <br\>
(e) ¿Cuántos de los registros en la variable ”zip Code” son valores no válidos? , es decir, contienen letras. <br\>
(f) Elimina los registros que no tengas un ”zip Code” válido, es decir contenga letras en los valores. <br\>
(g) ¿Cuántos de los registros en la variable ”genero” son valores no validos? , es decir, contienen letras. <br\>  
(h) Elimina los registros que no tengas un ”genero” válido, es decir contenga letras en los valores. <br\>
(i) Haz una limpieza de la variable ”name” , elimina caracteres especiales y todo debe estar en minúsculas. <br\>
(j) De la variable ”artist” selecciona solo al primer artista que aparezca en la lista además de eliminar caracteres especiales. <br\>
(k) Normaliza la variable ”genero” de tal forma que obtengas solo 8 categorı́as. <br\>
(l) Añade las siguientes columnas a tu tabla base: zip, lat, lng, city, state name uti- lizando la tabla zips practica. <br\>
(m) Convierte las variables ”lat” y ”lng” en tipo flotante y valida que la información sea consistente. <br\>
(n) De la variable ”city” y ”state” elimina los dı́gitos que se encuentran dentro de las cadenas de texto. <br\>  
(o) Crea una nueva variable llamada ”state” que este conformada por ”city”,”state name”. <br\>
(p) Los valores de la nueva variable ”state” modificalos de cierta forma que todas sean minúsculas y sin acentos. <br\>
(q) Convierte los valores en la variable ”release date” a tipo datetime , además haz el conteo de las que no tienen la estructura necesaria para ser convertida en datetime y elimina esos registros.
 <div>
     
***

In [114]:
import pandas as pd
import numpy as np
import unicodedata
import re

In [115]:
df = pd.read_csv("spotify_practica.csv")

/home/paramont/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


**A)** Realiza el etiquetado de las variables de acuerdo a su tipo de variable.  
    Etiquetas:  
    - c: continuas  
    - v: discretas  
    - d: fechas/tiempo  
    - t: texto

In [116]:
# Defininiendo generador para tener facil acceso al nombre de las 
# columnas con cada invocación.
gen_columns = (col for col in df.columns)

In [117]:
# Revisando tipo de dato y ejemplos de cada columna para decidir 
# su etiqueta.
col = next(gen_columns)
print(col)
print(type(df[col][0]))
df[col].head()

acousticness
<class 'numpy.float64'>


0    0.764
1    0.917
2    0.926
3    0.976
4    0.892
Name: acousticness, dtype: float64

In [118]:
# definiendo tipo de las columnas.
c_feats = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'popularity', 'speechiness', 'tempo', 'valence']
v_feats = ['explicit', 'key', 'mode', 'genero']
d_feats = ['duration_ms', 'release_date']
t_feats = ['artists', 'id', 'name', 'zip Code']

#creando listas de nuevas etiquetas.
c_feats_new=["c_"+x for x in c_feats]
v_feats_new=["v_"+x for x in v_feats]
d_feats_new=["d_"+x for x in d_feats]
t_feats_new=["t_"+x for x in t_feats]

#renombrando columnas
df.rename(columns=dict(zip(d_feats,d_feats_new)),inplace=True)
df.rename(columns=dict(zip(v_feats,v_feats_new)),inplace=True)
df.rename(columns=dict(zip(t_feats,t_feats_new)),inplace=True)
df.rename(columns=dict(zip(c_feats,c_feats_new)),inplace=True)

In [119]:
df.columns

Index(['c_acousticness', 't_artists', 'c_danceability', 'd_duration_ms',
       'c_energy', 'v_explicit', 't_id', 'c_instrumentalness', 'v_key',
       'c_liveness', 'c_loudness', 'v_mode', 't_name', 'c_popularity',
       'd_release_date', 'c_speechiness', 'c_tempo', 'c_valence', 't_zip Code',
       'v_genero'],
      dtype='object')

**B)** Eliminar duplicados.

In [120]:
# Revisando número de registros duplicados
df.duplicated().sum()

364

In [121]:
# Eliminando registros identicos
df.drop_duplicates(inplace=True)
# Eliminando registros si su id es identico
df.drop_duplicates(subset='t_id', inplace=True)
# Comprobando que ya no hay duplicados
df.duplicated().sum()

0

**C)** Completitud de las variables.

In [122]:
# Creando DataFrame para revisar completitud de cada columna.
comple=pd.DataFrame(df.isnull().sum())
comple=comple.rename(columns={"index":"columna",0:"total"})
comple["completitud"]=(1-comple['total']/df.shape[0])*100
# incluyendo etiquetas de las filas , que son las columnas, al df
comple.reset_index(inplace=True)
comple

,index,total,completitud
0,c_acousticness,0,100.000000
1,t_artists,0,100.000000
2,c_danceability,0,100.000000
3,d_duration_ms,0,100.000000
4,c_energy,0,100.000000
5,v_explicit,62883,62.990189
6,t_id,0,100.000000
7,c_instrumentalness,0,100.000000
8,v_key,35746,78.961679
9,c_liveness,0,100.000000


**D)** Elimina las variables que tengan el 20 % o m´as de valores ausentes.

In [123]:
# Obtenemos el nombre de las columnas cuya completitud es menor a 80
# extrayendo los valores de la columna index del df comple
index_to_drop = comple[comple["completitud"]<80]["index"].values
df.drop(columns = index_to_drop, inplace=True)

In [124]:
# Comprobando que se borraron v_key y v_explicit
"v_explicit" in df.columns

False

In [125]:
"v_key" in df.columns

False

**E)** ¿Cuántos de los registros en la variable ”zip Code” son valores no válidos? , es
decir, contienen letras.

In [126]:
# Usamos str.contains con una regex para obtener zip codes invalidos
invalid_zip_codes = df["t_zip Code"][df["t_zip Code"].str.contains('[A-Za-z]', na=False)]
# Con count tenemos el total de zip codes invalidos.
invalid_zip_codes.count()

4341

**F)** Elimina los registros que no tengas un ”zip Code” válido, es decir contenga letras
en los valores.

In [127]:
# Usamos drop con el parametro inplace que altere nuestro df
df.drop(invalid_zip_codes.index, inplace=True)

In [128]:
# Revisamos df
df.head()

,c_acousticness,t_artists,c_danceability,d_duration_ms,c_energy,t_id,c_instrumentalness,c_liveness,c_loudness,v_mode,t_name,c_popularity,d_release_date,c_speechiness,c_tempo,c_valence,t_zip Code,v_genero
24,0.000331,['Slayer'],0.296,400093,0.918,0dt3XQL7LjTNyizTXY00yD,0.178000,0.2530,-8.360,1,Crypts of Eternity,33.0,1985-05-26 00:00:00,0.0693,101.916,0.137,13126,NaN
25,0.329000,['Miguel Mateos - Zas'],0.548,290640,0.968,096vJncZZrwi4lLUoggD9y,0.172000,0.0759,-7.038,0,Mi Sombra En la Pared,45.0,1986-10-02 00:00:00,0.0778,174.593,0.786,14223,NaN
26,0.004680,['Paula Abdul'],0.731,231921,0.779,7xHYQboEmdZXWuXpJf9h30,0.001340,0.0525,-9.126,0,Knocked Out,32.0,1988-08-15 00:00:00,0.0454,116.233,0.954,84620,NaN
27,0.742000,['Henry Fiol'],0.708,305058,0.629,6M5z2Pca6OuN4l5n5kId3E,0.000003,0.0858,-10.358,0,Zumbale,46.0,1991-04-16 00:00:00,0.0351,95.451,0.915,22824,NaN
28,0.742000,['Eric Clapton'],0.598,216800,0.219,5tdEWfBGNX7a7zD78tUwLZ,0.056900,0.8790,-15.048,0,Walkin' Blues - Acoustic; Live at MTV Unplugge...,45.0,1992-08-15 00:00:00,0.0476,85.330,0.415,27519,NaN


In [129]:
# Reseteamos index
df.reset_index(drop=True, inplace=True)

**G)**¿Cuántos de los registros en la variable ”genero” son valores no validos? , es decir, contienen letras.

In [130]:
# Usamos str.contains nuevamente para excluir registros con letras.
invalid_genre = df[df["v_genero"].str.contains('(?:^[^\d])|(?:[a-zA-Z])', na=False)]["v_genero"]
invalid_genre.count()

10

**H)** Elimina los registros que no tengas un ”genero” válido, es decir contenga letras en
los valores.

In [131]:
# Eliminamos con parametro inplace=True
df.drop(invalid_genre.index, inplace=True)

In [132]:
#Reseteamos index
df.reset_index(drop=True, inplace=True)

**I)** Haz una limpieza de la variable ”name” , elimina caracteres especiales y todo debe
estar en minúsculas

In [133]:
# Usamos función definida en clase
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    """Cleans the text to facilitate its analysis, it changes accented letters to its quivalent without accents, removes also special characters

    Parameters
    ----------
    text : string
        String containing text
    pattern : str, optional
        Regular expession for keeping in the string, by default ``[^a-zA-Z0-9 ]``

    Returns
    -------
    cleaned_text : string
        Cleaned string

    Example
    -------
    >>> clean_text('¡Feliz año nuevo, México!')
    >>> u'feliz ano nuevo mexico'
    """
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().split())
    return cleaned_text

In [134]:
# Aplicamos la función a cada registro con lambda:
df["t_name"] = df["t_name"].map(lambda x:clean_text(x).replace("\n", ""))

**J)** De la variable ”artist” selecciona solo al primer artista que aparezca en la lista
además de eliminar caracteres especiales.

In [135]:
df["t_artists"][[0]]

0    ['Slayer']
Name: t_artists, dtype: object

In [136]:
# Ssamos la función previamente definida de clean_text junto 
# con una lambda
df["t_artists"] = df["t_artists"].map(lambda x:clean_text(x).replace("\n", ""))

In [137]:
# Comprobamos la limpiexa del string
df["t_artists"][[0]]

0    slayer
Name: t_artists, dtype: object

**K)** Normaliza la variable ”genero” de tal forma que obtengas solo 8 categorı́as

In [138]:
df["v_genero"].value_counts()

47.0    27167
20.0    24855
50.0    20144
12.0    18301
16.0    18054
21.0    17829
47.0     5812
20.0     5398
50.0     4247
21.0     3899
16.0     3843
12.0     3774
30.0     2653
49.0     2627
48.0     2565
30.0      579
49.0      566
48.0      551
3.0       448
2.0       285
1.0       147
3.0        84
2.0        65
1.0        23
Name: v_genero, dtype: int64

In [139]:
# Una primera forma para reducir las categorías de la columna,
# es pasando todos los registros al mismo tipo de dato: float
df["v_genero"] = df["v_genero"].map(float)
df["v_genero"].value_counts()

47.0    32979
20.0    30253
50.0    24391
12.0    22075
16.0    21897
21.0    21728
30.0     3232
49.0     3193
48.0     3116
3.0       532
2.0       350
1.0       170
Name: v_genero, dtype: int64

In [140]:
# Observamos 12 categorías. Juntaremos las 3 categorías con menos 
# frerecuencia en "otros" para reducir las categorías totales.
# Sin embargo, para que la categoría de "otros" siga siendo la de menor 
# frecuencia, no incluiremos el 4to genero de menor frecuencia.
# Para conservar que los generos sólo sean numéricos, definimos 99.0
# Como nuestra categoría "otros"
for i in df["v_genero"].value_counts().index[-3:]:
    print(i)
    print(type(i))
    df["v_genero"].replace({i:'99.0'}, inplace=True)

3.0
<class 'float'>
2.0
<class 'float'>
1.0
<class 'float'>


In [141]:
df["v_genero"].value_counts()

47.0    32979
20.0    30253
50.0    24391
12.0    22075
16.0    21897
21.0    21728
30.0     3232
49.0     3193
48.0     3116
99.0     1052
Name: v_genero, dtype: int64

**L)** Añade las siguientes columnas a tu tabla base: zip, lat, lng, city, state name uti-
lizando la tabla zips practica.

In [143]:
zips = pd.read_csv("zips_practica.csv")

In [144]:
# Hacemos el merge sobre el zip code, con un LEFT JOINT
df = df.merge(zips, left_on="t_zip Code", right_on="zip", how="left")

In [146]:
# comprobando el merge en registros cuyo campo city no es nulo
df[df["city"].notna()]

,c_acousticness,t_artists,c_danceability,d_duration_ms,c_energy,t_id,c_instrumentalness,c_liveness,c_loudness,v_mode,...,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone,c_lng
30483,0.78900,little jimmy dickens,0.471,150640,0.887000,5rgeemSaa9yETCkuoHrfLH,0.000345,0.2660,-3.734,1,...,6.9,5023.0,Cleburne,"{'05023':92.4,'05137':7.6}",Cleburne|Stone,05023|05137,False,False,America/Chicago,NaN
30484,0.99500,sonny clark,0.405,316893,0.195000,5rhqW8QJogXTGWi4DEap3Y,0.668000,0.3640,-23.019,1,...,160.2,36091.0,Saratoga,{'36091':100},Saratoga,36091,False,False,America/New_York,NaN
30485,0.97700,julius larosa,0.436,165253,0.004580,5ubalnFpVACOrtetnZiFKO,0.000006,0.1060,-26.393,1,...,331.5,22083.0,Richland,{'22083':100},Richland,22083,False,False,America/Chicago,NaN
30486,0.83900,xavier cugat his orchestra,0.758,162933,0.548000,5urIfyjGu6S2JVMY4pa6Yr,0.895000,0.3980,-11.589,0,...,4.0,19151.0,Pocahontas,"{'19147':2.71,'19151':97.29}",Pocahontas|Palo Alto,19151|19147,False,False,America/Chicago,NaN
30487,0.83600,charles mingus john laporta,0.274,195200,0.178000,5xc4Wm7b5Hrl4fACGs1ZWv,0.920000,0.1090,-11.946,0,...,2.1,27149.0,Stevens,"{'27149':97.21,'27151':2.79}",Stevens|Swift,27149|27151,False,False,America/Chicago,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161550,0.00766,cock sparrer,0.307,177893,0.895000,1MG6dTIs1niCx6wJCS9YiP,0.000000,0.0317,-4.745,1,...,15.4,29097.0,Jasper,"{'29097':81.93,'29109':5.15,'29145':12.92}",Jasper|Lawrence|Newton,29097|29109|29145,False,False,America/Chicago,NaN
161551,0.29300,bad brains,0.420,45000,0.000281,2825UQ58tG56ozABVD8bwf,0.442000,0.7490,-15.637,1,...,69.6,36051.0,Livingston,"{'36051':93.99,'36069':6.01}",Livingston|Ontario,36051|36069,False,False,America/New_York,NaN
161552,0.01580,talking heads,0.570,504840,0.897000,7tZDm498uu16woGTlK1V4p,0.013700,0.8550,-6.692,0,...,21.7,51027.0,Buchanan,{'51027':100},Buchanan,51027,False,False,America/New_York,NaN
161553,0.07230,hank williams jr,0.617,156107,0.752000,1dh1DBjWKUSTtycdn5kJwo,0.000007,0.2020,-5.667,1,...,104.1,13313.0,Whitfield,"{'13047':37.9,'13313':62.1}",Whitfield|Catoosa,13313|13047,False,False,America/New_York,NaN


**M)** Convierte las variables ”lat” y ”lng” en tipo flotante y valida que la información
sea consistente.

In [150]:
# Convirtiendo a tipo float
df["lat"].astype(float)
df["lng"].astype(float)
df[df["lat"].notna()][["lat", "lng"]]

,lat,lng
30483,35.61819,-91.95341
30484,43.00248,-73.86800
30485,32.48731,-91.85714
30486,42.84799,-94.54307
30487,45.49685,-96.06945
...,...,...
161550,37.09740,-94.11430
161551,42.81175,-77.63597
161552,37.21374,-81.99169
161553,34.86684,-85.04077


In [152]:
# Los valores de latitud deben ubicarse dento de los -90 a 90 °.
df["lat"].describe()

count    131072.000000
mean         38.850304
std           5.337935
min          17.966120
25%          35.435820
50%          39.509060
75%          42.129260
max          71.273740
Name: lat, dtype: float64

Los valores máximos y mínimos de la latitud están dentro del rango esperado.

In [153]:
# Mientras que los valores de longitud se deben ubicar entre -180 y 180
df["lng"].describe()

count    131072.000000
mean        -91.155202
std          16.076207
min        -331.846760
25%         -97.265630
50%         -88.212620
75%         -80.200890
max         -65.290170
Name: lng, dtype: float64

Existen valores de longitud por debajo del límite inferior del rango permitido (-180). Sin embargo, la corrección de dicho valor debe ser algo que requiera de mejores herramientas, ya que el valor erroneo de la longitud parece no tener una relación simple con el valor correcto.
  
Por ejemplo, la longitud de Black Creek, NY es 3.31 veces mayor a la correcto, mientras que la de Spring Valley, OH es 3.15 veces mayor.

**N)** De la variable ”city” y ”state” elimina los dı́gitos que se encuentran dentro de las
cadenas de texto.

In [155]:
# Usamos str.replace con una regex
df["city"] = df["city"].str.replace('\d+','')

<ipython-input-155-98b57db62a43>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["city"] = df["city"].str.replace('\d+','')


In [156]:
# comprobamos
df[df["city"].notna()]["city"]

30483           Drasco
30484     Ballston Spa
30485            Start
30486            Rolfe
30487          Alberta
              ...     
161550        Sarcoxie
161551         Livonia
161552         Oakwood
161553     Tunnel Hill
161554     Bella Vista
Name: city, Length: 131072, dtype: object

In [160]:
df["state_name"] = df["state_name"].str.replace('\d+','')
df[df["state_name"].notna()]["state_name"]

<ipython-input-160-b4ae8c2bd480>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["state_name"] = df["state_name"].str.replace('\d+','')


30483       Arkansas
30484       New York
30485      Louisiana
30486           Iowa
30487      Minnesota
             ...    
161550      Missouri
161551      New York
161552      Virginia
161553       Georgia
161554    California
Name: state_name, Length: 131072, dtype: object

**O)** Crea una nueva variable llamada ”state” que este conformada por ”city”,”state name”.

In [161]:
# Usamos str.cat
df["state"] = df["state_name"].str.cat(df["city"], sep=' ')

In [162]:
# Comprobamos
df[df["state"].notna()]["state"]

30483            Arkansas Drasco
30484      New York Ballston Spa
30485            Louisiana Start
30486                 Iowa Rolfe
30487          Minnesota Alberta
                   ...          
161550         Missouri Sarcoxie
161551          New York Livonia
161552          Virginia Oakwood
161553       Georgia Tunnel Hill
161554    California Bella Vista
Name: state, Length: 131072, dtype: object

**P)** Los valores de la nueva variable ”state” modificalos de cierta forma que todas sean
minúsculas y sin acentos.

In [164]:
# Usamos la función clean_text previamente definida
df["state"] = df[df["state"].notna()]["state"].map(lambda x:clean_text(x).replace("\n", ""))

In [165]:
df[df["state"].notna()]["state"]

30483            arkansas drasco
30484      new york ballston spa
30485            louisiana start
30486                 iowa rolfe
30487          minnesota alberta
                   ...          
161550         missouri sarcoxie
161551          new york livonia
161552          virginia oakwood
161553       georgia tunnel hill
161554    california bella vista
Name: state, Length: 131072, dtype: object

**Q)** Convierte los valores en la variable ”release date” a tipo datetime , además haz el
conteo de las que no tienen la estructura necesaria para ser convertida en datetime
y elimina esos registros.

In [166]:
# Usamos pd.to_date
df["d_release_date"] = pd.to_datetime(df["d_release_date"], errors='coerce')

In [167]:
df["d_release_date"] 

0        1985-05-26
1        1986-10-02
2        1988-08-15
3        1991-04-16
4        1992-08-15
            ...    
165553   1935-01-25
165554   1935-02-22
165555   1935-09-02
165556   1935-07-22
165557   1935-07-15
Name: d_release_date, Length: 165558, dtype: datetime64[ns]

In [168]:
df[df["d_release_date"].isna()].value_counts()

Series([], dtype: int64)

Como en usamos el parametro error="coerce" en to_datetime, hubieramos obtenido valores nulos en los registros que no hubiesen podido ser transformados por la función. Sin embargo, no tenemos ningún valor nulo, por lo que no debió de haber habido registros con un formato que no pudiera ser interpretado como un dato datetime.

***

### 2 De la tabla tratada obten los siguientes datos:  
  
<div style="text-align: justify"> 
(a) Obten un DataFrame donde se muestre el conteo de registros por género <br\>
(b) Genera una nueva variable que se llame ”duration minutos” que sea el valor de la variable ”duration ms” en minutos<br\>
(c) ¿Cuáles son las 10 canciones más populares?<br\>
(d) ¿Cúal es el promedio de duración en minutos y milisegundos?<br\>
(e) ¿Cuál es el promedio y conteo de ”energy” por ”genero”?<br\>
(f) ¿Qué canción tiene el ”loudness” más bajo y cúal es la canción que tiene el ”loudness” más alto?<br\>
(g) Obtenga los 10 percentiles de las variables continuas<br\>
<div>
    
 ***

**A)** Obten un DataFrame donde se muestre el conteo de registros por género.

In [172]:
# Creamos un dataframe con el uso de value_counts()
df_genre_freq = pd.DataFrame(df["v_genero"].value_counts())

In [173]:
# Renombramos columnas
df_genre_freq.reset_index().rename(columns={"index":"genre", "v_genero":"frequency"})

,genre,frequency
0,47.00000,32979
1,20.00000,30253
2,50.00000,24391
3,12.00000,22075
4,16.00000,21897
5,21.00000,21728
6,30.00000,3232
7,49.00000,3193
8,48.00000,3116
9,99.0,1052


**B)** Genera una nueva variable que se llame ”duration minutos” que sea el valor de la
variable ”duration ms” en minutos

In [174]:
#Pasaremos los datos a un dato datetime
df["duration_minutes"] = pd.to_timedelta(df["d_duration_ms"], unit='ms')

In [175]:
df["duration_minutes"]

0        0 days 00:06:40.093000
1        0 days 00:04:50.640000
2        0 days 00:03:51.921000
3        0 days 00:05:05.058000
4        0 days 00:03:36.800000
                  ...          
165553   0 days 00:01:41.745000
165554   0 days 00:01:53.100000
165555          0 days 00:01:48
165556   0 days 00:03:28.600000
165557   0 days 00:04:54.213000
Name: duration_minutes, Length: 165558, dtype: timedelta64[ns]

**C)** ¿Cuáles son las 10 canciones más populares?

In [177]:
# Haremos una subcolsulta con el uso de sort e index slicing para 
# obtener las canciones con mayor puntaje de popularidad
top_10_index = df["c_popularity"].sort_values(ascending=False).index[:10]
df["t_name"][top_10_index]

84162                                    blinding lights
84160                          rockstar feat roddy ricch
84169    death bed coffee for your head feat beabadoobee
84161                                         the scotts
84073                    ily i love you baby feat emilee
84065                                         supalonely
84166                                       toosie slide
84061                                            the box
84172                      rain on me with ariana grande
84079                                       dance monkey
Name: t_name, dtype: object

**D)**¿Cúal es el promedio de duración en minutos y milisegundos?

In [178]:
#En milisegundos
df["d_duration_ms"].mean()

231400.2148612571

In [179]:
# En minutos
df["duration_minutes"].mean()

Timedelta('0 days 00:03:51.400214861')

**E)** ¿Cuál es el promedio y conteo de ”energy” por ”genero”?

In [180]:
# Agrupamos por genero y aplicamos funciones de agregación
df[["v_genero", "c_energy"]].groupby("v_genero").agg(["mean", "count"])

c_energy       
              mean  count
v_genero                 
12.0      0.493339  22075
16.0      0.488489  21897
20.0      0.488760  30253
21.0      0.489807  21728
30.0      0.488098   3232
47.0      0.491360  32979
48.0      0.493174   3116
49.0      0.489755   3193
50.0      0.492117  24391
99.0      0.480530   1052

**F)** ¿Qué canción tiene el ”loudness” más bajo y cúal es la canción que tiene el ”loud- ness” más alto?

In [181]:
# mínimo
min_loudness = df[df["c_loudness"]==df["c_loudness"].min()]
min_loudness

,c_acousticness,t_artists,c_danceability,d_duration_ms,c_energy,t_id,c_instrumentalness,c_liveness,c_loudness,v_mode,...,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone,c_lng,state,duration_minutes
38695,0.0,connie francis,0.0,179466,0.0,0zr7DJGTPUfAUmjM7crmt2,0.0,0.0,-60.0,0,...,Chatham,"{'37037':98.4,'37151':1.6}",Chatham|Randolph,37037|37151,False,False,America/New_York,NaN,north carolina siler city,0 days 00:02:59.466000
45210,0.0,sarah vaughan,0.0,6467,0.0,3lRVIn6D6EUbvkOgPZAU1H,0.0,0.0,-60.0,0,...,Jefferson,{'48245':100},Jefferson,48245,False,False,America/Chicago,NaN,texas groves,0 days 00:00:06.467000
57590,0.0,robert earl keen,0.0,60372,0.0,0o12mLSQuXFgsh4e2Kc4e5,0.0,0.0,-60.0,0,...,Mono,{'06051':100},Mono,06051,False,False,America/Los_Angeles,NaN,california lee vining,0 days 00:01:00.372000
58472,0.0,atlas fret,0.0,132827,0.0,7cctPQS83y620UQtMd1ilL,0.0,0.0,-60.0,0,...,Pike,{'18125':100},Pike,18125,False,False,America/Indiana/Petersburg,NaN,indiana stendal,0 days 00:02:12.827000
61061,0.0,connie francis,0.0,253719,0.0,5KAJv7Bceihn1frqElloIb,0.0,0.0,-60.0,0,...,New London,{'09011':100},New London,09011,False,False,America/New_York,NaN,connecticut niantic,0 days 00:04:13.719000
95180,0.0,benny goodman,0.0,5991,0.0,3IcXTeq9O2dpsSXsDj9naH,0.0,0.0,-60.0,0,...,Ontario,{'36069':100},Ontario,36069,False,False,America/New_York,NaN,new york shortsville,0 days 00:00:05.991000
104469,0.0,benny goodman,0.0,6362,0.0,523qs4UcGlQ6ycdha1VGqs,0.0,0.0,-60.0,0,...,Passaic,{'34031':100},Passaic,34031,False,False,America/New_York,NaN,new jersey wayne,0 days 00:00:06.362000
132931,0.0,future rapper,0.0,420000,0.0,0Rd7eiAZGayLT8TmrVpQzG,0.0,0.0,-60.0,0,...,Allegheny,{'42003':100},Allegheny,42003,False,False,America/New_York,NaN,pennsylvania pittsburgh,0 days 00:07:00
141496,0.0,sarah vaughan,0.0,5108,0.0,0hr9kRUi2X4MXc72A4VxG4,0.0,0.0,-60.0,0,...,Sacramento,{'06067':100},Sacramento,06067,False,False,America/Los_Angeles,NaN,california sacramento,0 days 00:00:05.108000


In [182]:
# Máximo
max_loudness = df[df["c_loudness"]==df["c_loudness"].max()]
max_loudness

,c_acousticness,t_artists,c_danceability,d_duration_ms,c_energy,t_id,c_instrumentalness,c_liveness,c_loudness,v_mode,...,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone,c_lng,state,duration_minutes
104268,0.000047,apocolothoth,0.0,152476,0.114,050FXMyCrQJG01AT55Jvk1,0.998,0.578,3.855,0,...,Campbell,{'47013':100},Campbell,47013,False,False,America/New_York,NaN,tennessee la follette,0 days 00:02:32.476000


**G**) Obtenga los 10 percentiles de las variables continuas

In [185]:
# Usamos un loop junto con filter para encontrar los cuantiles de cada
# variable continua
for column in df.filter(like="c_"):
    print(column)
    print(df[column].quantile([x/10 for x in range(1,11)]))
    print("\n")

c_acousticness
0.1    0.00871
0.2    0.05190
0.3    0.14310
0.4    0.28900
0.5    0.48300
0.6    0.68000
0.7    0.82700
0.8    0.93200
0.9    0.98300
1.0    0.99600
Name: c_acousticness, dtype: float64


c_danceability
0.1    0.299
0.2    0.385
0.3    0.448
0.4    0.501
0.5    0.549
0.6    0.596
0.7    0.643
0.8    0.696
0.9    0.761
1.0    0.988
Name: c_danceability, dtype: float64


c_energy
0.1    0.131
0.2    0.222
0.3    0.307
0.4    0.396
0.5    0.483
0.6    0.572
0.7    0.666
0.8    0.761
0.9    0.868
1.0    1.000
Name: c_energy, dtype: float64


c_instrumentalness
0.1    0.000000
0.2    0.000000
0.3    0.000002
0.4    0.000020
0.5    0.000192
0.6    0.001940
0.7    0.022700
0.8    0.275000
0.9    0.829000
1.0    1.000000
Name: c_instrumentalness, dtype: float64


c_liveness
0.1    0.0729
0.2    0.0911
0.3    0.1050
0.4    0.1160
0.5    0.1350
0.6    0.1670
0.7    0.2230
0.8    0.3050
0.9    0.4070
1.0    1.0000
Name: c_liveness, dtype: float64


c_loudness
0.1   -18.8683
0.2   